# Predicting Graduation Rates Exploratory Data Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
#conda install -c conda-forge cufflinks-py
#conda install plotly
import ipywidgets as wg
from IPython.display import display
import pandas as pd 
import numpy as np
import cufflinks as cf
import chart_studio.plotly as py
import plotly.express as px
%matplotlib inline
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import inspect
import seaborn as sns
init_notebook_mode(connected=True)
cf.go_offline()

In [3]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.compose import ColumnTransformer
pd.options.display.max_columns = 200
pd.options.display.max_rows = 272

In [4]:
MAX_DOLLAR = 10000000000
REV_DOLLAR = 1500000000
EXP_DOLLAR = 1000000000

In [5]:
df = pd.read_csv("data/cleaned_df.csv", index_col=["Institution Name"])

In [6]:
schema = pd.read_csv("data/schema.csv")

In [7]:
df.drop(["Unnamed: 0", "UnitID"], axis=1,inplace=True)

In [8]:
df.head()

,Core_Revenues,Tuition_And_Fees,Government_Grants,Private_Gifts,Investment_Return,Sales_And_Services,Other_Revenues,Tuition_And_Fees_As_Dollar_Amount,Government_Grants_As_Dollar_Amount,Private_Gifts_As_Dollar_Amount,Investment_Return_As_Dollar_Amount,Sales_And_Services_As_Dollar_Amount,Other_Revenues_As_Dollar_Amount,Core_Expenses,Instruction_Expenses,Research_Expenses,Public_Service_Expenses,Academic_Support_Expenses,Student_Service_Expenses,Institutional_Support_Expenses,Other_Core_Expenses,Instruction_Expenses_As_Dollar_Amount,Research_Expenses_As_Dollar_Amount,Public_Service_Expenses_As_Dollar_Amount,Academic_Support_Expenses_As_Dollar_Amount,Student_Service_Expenses_As_Dollar_Amount,Institutional_Support_Expenses_As_Dollar_Amount,Other_Core_Expenses_As_Dollar_Amount,Grand total instructional_staff,Grand total men instructional_staff,Grand total women instructional_staff,American Indian or Alaska Native total instructional_staff,American Indian or Alaska Native men instructional_staff,American Indian or Alaska Native women instructional_staff,Asian total instructional_staff,Asian men instructional_staff,Asian women instructional_staff,Black or African American total instructional_staff,Black or African American men instructional_staff,Black or African American women instructional_staff,Hispanic or Latino total instructional_staff,Hispanic or Latino men instructional_staff,Hispanic or Latino women instructional_staff,Native Hawaiian or Other Pacific Islander total instructional_staff,Native Hawaiian or Other Pacific Islander men instructional_staff,Native Hawaiian or Other Pacific Islander women instructional_staff,White total instructional_staff,White men instructional_staff,White women instructional_staff,Two or more races total instructional_staff,Two or more races men instructional_staff,Two or more races women instructional_staff,Race/ethnicity unknown total instructional_staff,Race/ethnicity unknown men instructional_staff,Race/ethnicity unknown women instructional_staff,Nonresident alien total instructional_staff,Nonresident alien men instructional_staff,Nonresident alien women instructional_staff,Grand total men instructional_staff_as_percentage,Grand total women instructional_staff_as_percentage,American Indian or Alaska Native total instructional_staff_as_percentage,American Indian or Alaska Native men instructional_staff_as_percentage,American Indian or Alaska Native women instructional_staff_as_percentage,Asian total instructional_staff_as_percentage,Asian men instructional_staff_as_percentage,Asian women instructional_staff_as_percentage,Black or African American total instructional_staff_as_percentage,Black or African American men instructional_staff_as_percentage,Black or African American women instructional_staff_as_percentage,Hispanic or Latino total instructional_staff_as_percentage,Hispanic or Latino men instructional_staff_as_percentage,Hispanic or Latino women instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander total instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander men instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander women instructional_staff_as_percentage,White total instructional_staff_as_percentage,White men instructional_staff_as_percentage,White women instructional_staff_as_percentage,Two or more races total instructional_staff_as_percentage,Two or more races men instructional_staff_as_percentage,Two or more races women instructional_staff_as_percentage,Race/ethnicity unknown total instructional_staff_as_percentage,Race/ethnicity unknown men instructional_staff_as_percentage,Race/ethnicity unknown women instructional_staff_as_percentage,Nonresident alien total instructional_staff_as_percentage,Nonresident alien men instructional_staff_as_percentage,Nonresident alien women instructional_staff_as_percentage,Percent of full-time first-time undergraduates awarded any financial aid (SFA1819),Percent of full-time first-time und

In [9]:
universities_list = list(df.index)

In [10]:
# grab specific University
inspect_list = []
search_for = ["Clemson", "Clem", "Asheville", " of Delaware"]
for i in universities_list:
    for j in search_for:
        if j in i:
            inspect_list.append(i)
            break
    

In [11]:
inspect_list

['Clemson University',
 'University of Delaware',
 'University of North Carolina at Asheville']

In [12]:
target = df.iloc[:, -14:]

In [13]:
target_names = list(target.columns)

In [14]:
features = df.iloc[:, :-14]

In [15]:
features.shape

(2315, 134)

# Visualizing Numeric Features

In [16]:
numeric_df = df.select_dtypes(include=["int", "float"])

In [17]:
summary = numeric_df.iloc[:,[0,13]].columns

In [18]:
num_names = list(numeric_df.columns)

In [19]:
rev_dist_widget = wg.Dropdown(
    options=num_names[1:7],
    value=num_names[1],
    description="Feature:",
    disabled=False
)

target_widget = wg.Dropdown(
    options=target_names,
    value=target_names[0],
    description="Target",
    disabled=False
)

ui = wg.HBox([rev_dist_widget, target_widget])

def revenues_dist(features, target):
    
    title = str(features) + " vs. " + str(target)
    fig = px.scatter(
        df, x=features, y=target, range_x=[-1,101], range_y=[-1,101],
        title=title, trendline='ols'
    )
    
    results = px.get_trendline_results(fig)
    display(
        fig.show(), results.iloc[0]['px_fit_results'].summary()
    )
    
categorical_output = wg.interactive_output(revenues_dist, {'features':rev_dist_widget, 'target':target_widget})

display(ui, categorical_output)


Output()

# Revenues as Dollar Amount

In [30]:
rev_dist_widget = wg.Dropdown(
    options=num_names[7:13],
    value=num_names[7],
    description="Feature:",
    disabled=False
)

rev_target_widget = wg.Dropdown(
    options=target_names,
    value=target_names[0],
    description="Target",
    disabled=False
)

ui = wg.HBox([rev_dist_widget, rev_target_widget])

def revenues_dollar(features, target):
    
    title = str(features) + " vs. " + str(target)
    fig = px.scatter(
        df, x=features, y=target, trendline='ols', 
        range_x=[0,REV_DOLLAR], range_y=[-1,101],
        title=title, height=700, 
    )
    results = px.get_trendline_results(fig)
    display(
        fig.show(), results.iloc[0]['px_fit_results'].summary()
    )
    
categorical_output = wg.interactive_output(revenues_dollar, {'features':rev_dist_widget, 'target':rev_target_widget})

display(ui, categorical_output)


Output()

# Expenses Distribution

In [21]:
exp_dist_widget = wg.Dropdown(
    options=num_names[14:19],
    value=num_names[14],
    description="Feature:",
    disabled=False
)

exp_target_widget = wg.Dropdown(
    options=target_names,
    value=target_names[0],
    description="Target",
    disabled=False
)

ui = wg.HBox([exp_dist_widget, exp_target_widget])

def expenses_dist(features, target):
    
    title = str(features) + " vs. " + str(target)
    fig = px.scatter(
            df, x=features, y=target, range_x=[-1,101], range_y=[-1,101],
            title=title, trendline='ols', height=700
    )
    results = px.get_trendline_results(fig)
    display(
        fig.show(), results.iloc[0]["px_fit_results"].summary()
    )
    
categorical_output = wg.interactive_output(expenses_dist, {'features':exp_dist_widget, 'target':exp_target_widget})

display(ui, categorical_output)

Output()

# Expenses as Dollar Amount

In [22]:
exp_dollar_widget = wg.Dropdown(
    options=num_names[21:27],
    value=num_names[21],
    description="Feature:",
    disabled=False
)

exp_target_widget = wg.Dropdown(
    options=target_names,
    value=target_names[0],
    description="Target",
    disabled=False
)

ui = wg.HBox([exp_dollar_widget, exp_target_widget])

def expenses_dollar(features, target):
    
    title = str(features) + " vs. " + str(target)
    fig = px.scatter(
            df, x=features, y=target, range_x=[-1,EXP_DOLLAR], range_y=[-1,101],
            title=title, trendline='ols', height=700
    )
    results = px.get_trendline_results(fig)
    display(
        fig.show(), results.iloc[0]["px_fit_results"].summary()
    )
    
categorical_output = wg.interactive_output(expenses_dollar, {'features':exp_dollar_widget, 'target':exp_target_widget})

display(ui, categorical_output)

Output()

# Finances Summary

In [23]:
summary_widget = wg.Dropdown(
    options=summary,
    description="Feature:",
    disabled=False
)

exp_target_widget = wg.Dropdown(
    options=target_names,
    value=target_names[0],
    description="Target",
    disabled=False
)

ui = wg.HBox([summary_widget, exp_target_widget])

def finances(features, target):
    
    title = str(features) + " vs. " + str(target)
    fig = px.scatter(
            df, x=features, y=target, range_x=[-1,MAX_DOLLAR], range_y=[-1,101],
            title=title, trendline='ols', height=700
    )
    results = px.get_trendline_results(fig)
    display(
        fig.show(), results.iloc[0]["px_fit_results"].summary()
    )
    
categorical_output = wg.interactive_output(finances, {'features':summary_widget, 'target':exp_target_widget})

display(ui, categorical_output)

Output()

# Visualizing Categorical Features

In [24]:
df.drop("City location of institution (HD2019)", axis=1, inplace=True)

In [25]:
df1 = pd.get_dummies(df, drop_first=True)

In [26]:
pd.DataFrame(df.isna().sum())

,0
Core_Revenues,4
Tuition_And_Fees,4
Government_Grants,4
Private_Gifts,4
Investment_Return,4
Sales_And_Services,4
Other_Revenues,4
Tuition_And_Fees_As_Dollar_Amount,4
Government_Grants_As_Dollar_Amount,4
Private_Gifts_As_Dollar_Amount,4


In [27]:
category = []
for i in df1.columns:
    if "Historically Black" in i:
        category.append(i)

In [43]:
grad_rates = df1.filter(regex='Graduation').columns[0]